In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

import hit_prediction_code.common as common
from hit_prediction_code.dataloaders import EssentiaLoader

dataset_name = 'msd_bb_balanced_essentia'
dataloader = EssentiaLoader(
    dataset_path='../data/processed/'+dataset_name+ '.pickle',
    features=[
        ('hl', 'wide'),
        ('year', 'wide'),
    ],
    label='peak',
    nan_value=150,
)

data, labels = dataloader.load()

In [ ]:
features = map(lambda x: list(zip(x[0][0], x[1][0])), zip(dataloader.configuration['features_list'],dataloader.feature_indices))

correlation = []
for feature_group in features:
    for feature, index in feature_group:
        try:
            correlation.append({'feature': str(feature), 'pearson': scipy.stats.pearsonr(data[:, index], labels)[0], 'kendall': scipy.stats.kendalltau(data[:, index], labels)[0], 'spearman': scipy.stats.spearmanr(data[:, index], labels)[0]})
        except TypeError:
            pass

correlation = pd.DataFrame(correlation).dropna()

correlation.plot.bar(x='feature', grid=True, figsize=(30,10))
plt.savefig('../results/correlation_' + dataset_name + '.pdf')